In [1]:
import pandas as pd

customer_train = pd.read_csv('/home/jovyan/MGTA452_Final-1/customer_train.csv')
customer_test = pd.read_csv('/home/jovyan/MGTA452_Final-1/customer_test.csv')
trip_train = pd.read_csv('/home/jovyan/MGTA452_Final-1/trip_train.csv')
trip_test = pd.read_csv('/home/jovyan/MGTA452_Final-1/trip_test.csv')

summary = {
    "customer_train": {
        "num_records": customer_train.shape[0],
        "columns": customer_train.columns.tolist(),
        "preview": customer_train.head()
    },
    "customer_test": {
        "num_records": customer_test.shape[0],
        "columns": customer_test.columns.tolist(),
        "preview": customer_test.head()
    },
    "trip_train": {
        "num_records": trip_train.shape[0],
        "columns": trip_train.columns.tolist(),
        "preview": trip_train.head()
    },
    "trip_test": {
        "num_records": trip_test.shape[0],
        "columns": trip_test.columns.tolist(),
        "preview": trip_test.head()
    }
}

summary


{'customer_train': {'num_records': 700,
  'columns': ['CustomerID', 'Age', 'Gender', 'Income'],
  'preview':    CustomerID  Age  Gender        Income
  0         656   58       0  44508.143303
  1         955   67       1  48992.269646
  2         916   66       0  63177.578696
  3          25   67       0  39584.587489
  4         735   21       0  39124.256687},
 'customer_test': {'num_records': 300,
  'columns': ['CustomerID', 'Age', 'Gender', 'Income'],
  'preview':    CustomerID  Age  Gender        Income
  0         260   59       1  52759.238749
  1         914   69       1  35171.285170
  2         931   68       0  53297.369235
  3         498   28       1  67622.548744
  4         949   37       1  27496.347582},
 'trip_train': {'num_records': 13952,
  'columns': ['CustomerID', 'Season', 'Brand', 'Dining', 'Spa', 'Rooms'],
  'preview':    CustomerID  Season  Brand      Dining         Spa       Rooms
  0           3       1      0  226.574587  346.373250  396.653659
  1       

In [2]:
num_female_customers = customer_train[customer_train['Gender'] == 1].shape[0]
num_female_customers


352

In [3]:
average_age_female = customer_train[customer_train['Gender'] == 1]['Age'].mean()

average_age_female_rounded = round(average_age_female)
average_age_female_rounded


44

In [4]:
average_income_female = customer_train[customer_train['Gender'] == 1]['Income'].mean()

average_income_female_rounded = round(average_income_female)
average_income_female_rounded


51411

In [5]:
average_dining_high_season = trip_train[trip_train['Season'] == 1]['Dining'].mean()

average_dining_high_season_rounded = round(average_dining_high_season)
average_dining_high_season_rounded


273

In [6]:
average_room_upscale_low_season = trip_train[(trip_train['Season'] == 0) & (trip_train['Brand'] == 1)]['Rooms'].mean()

average_room_discount_low_season = trip_train[(trip_train['Season'] == 0) & (trip_train['Brand'] == 0)]['Rooms'].mean()

difference_room_spending = average_room_upscale_low_season - average_room_discount_low_season

difference_room_spending_rounded = round(difference_room_spending)
difference_room_spending_rounded


24

In [7]:
merged_data = pd.merge(trip_train, customer_train, on='CustomerID')

average_room_spending_female = merged_data[merged_data['Gender'] == 1]['Rooms'].mean()

average_room_spending_female_rounded = round(average_room_spending_female)
average_room_spending_female_rounded


497

In [8]:
average_total_spending_male_upscale_high = merged_data[(merged_data['Gender'] == 0) & (merged_data['Brand'] == 1) & (merged_data['Season'] == 1)].eval('Dining + Spa + Rooms').mean()

average_total_spending_male_upscale_high_rounded = round(average_total_spending_male_upscale_high)
average_total_spending_male_upscale_high_rounded


992

In [9]:
segment_a_criteria = (customer_train['Gender'] == 1) & (customer_train['Age'] == 50) & (customer_train['Income'] >= 60000)

segment_a_customers = customer_train[segment_a_criteria]

segment_a_trip_data = pd.merge(segment_a_customers, trip_train, on='CustomerID')

segment_a_upscale_high_season = segment_a_trip_data[(segment_a_trip_data['Brand'] == 1) & (segment_a_trip_data['Season'] == 1)]

num_customers_segment_a = segment_a_upscale_high_season['CustomerID'].nunique()
num_customers_segment_a


1

In [10]:
segment_b_criteria = (customer_train['Gender'] == 1) & (customer_train['Age'] == 30) & (customer_train['Income'] >= 50000)

segment_b_customers = customer_train[segment_b_criteria]

segment_b_trip_data = pd.merge(segment_b_customers, trip_train, on='CustomerID')

segment_b_upscale_high_season = segment_b_trip_data[(segment_b_trip_data['Brand'] == 1) & (segment_b_trip_data['Season'] == 1)]

num_customers_segment_b = segment_b_upscale_high_season['CustomerID'].nunique()
num_customers_segment_b


1


I would say: "Comparing the spending of just one customer in each segment (A and B) may not provide a reliable basis for decision-making. This approach risks drawing conclusions from an extremely limited sample size, which might not be representative of the broader segment. It would be more insightful to analyze a larger, more diverse sample from each segment to accurately assess their average spending and potential value to the hotel."

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt

train_data = pd.merge(trip_train, customer_train, on='CustomerID')
test_data = pd.merge(trip_test, customer_test, on='CustomerID')

features = ['Season', 'Brand', 'Age', 'Income', 'Gender']
target_dining = 'Dining'

model_dining = LinearRegression()
model_dining.fit(train_data[features], train_data[target_dining])

# Predict on test data
predictions_dining = model_dining.predict(test_data[features])

rmse_dining = sqrt(mean_squared_error(test_data[target_dining], predictions_dining))
rmse_dining_rounded = round(rmse_dining)
rmse_dining_rounded


89

In [12]:
target_spa = 'Spa'
target_rooms = 'Rooms'

model_spa = LinearRegression()
model_spa.fit(train_data[features], train_data[target_spa])

predictions_spa = model_spa.predict(test_data[features])

model_rooms = LinearRegression()
model_rooms.fit(train_data[features], train_data[target_rooms])

predictions_rooms = model_rooms.predict(test_data[features])

total_predicted_spending = predictions_dining + predictions_spa + predictions_rooms

total_actual_spending = test_data[target_dining] + test_data[target_spa] + test_data[target_rooms]

rmse_total = sqrt(mean_squared_error(total_actual_spending, total_predicted_spending))
rmse_total_rounded = round(rmse_total)
rmse_total_rounded


266

In [13]:
train_data['TotalSpending'] = train_data[target_dining] + train_data[target_spa] + train_data[target_rooms]
test_data['TotalSpending'] = test_data[target_dining] + test_data[target_spa] + test_data[target_rooms]

target_total_spending = 'TotalSpending'

model_total_spending = LinearRegression()
model_total_spending.fit(train_data[features], train_data[target_total_spending])

predictions_total_spending = model_total_spending.predict(test_data[features])

rmse_total_spending = sqrt(mean_squared_error(test_data[target_total_spending], predictions_total_spending))
rmse_total_spending_rounded = round(rmse_total_spending)
rmse_total_spending_rounded


266

The RMSE obtained in both Question 12 and Question 13 is the same, at 266 when rounded to the nearest integer. In Question 12, total spending was predicted by separately modeling and summing the predictions for Dining, Spa, and Rooms spending. In contrast, Question 13 involved directly modeling total spending as a single output. The identical RMSE in both approaches suggests that the aggregated error from the individual predictions of Dining, Spa, and Rooms in Question 12 closely matches the error from the direct prediction of total spending in Question 13. This outcome can occur when the errors in individual models (Dining, Spa, Rooms) are not significantly large or are possibly offsetting each other when summed, leading to a similar overall prediction accuracy for total spending as achieved by a single model predicting the total directly.

In [14]:
profit_margin_dining = 0.20 
profit_margin_spa = 0.35     
profit_margin_rooms = 0.25   

In [15]:
segment_1_features = pd.DataFrame({
    'Season': [1], 
    'Brand': [1], 
    'Age': [30],
    'Income': [50000],
    'Gender': [1] 
})

predicted_dining_1 = model_dining.predict(segment_1_features)[0]
predicted_spa_1 = model_spa.predict(segment_1_features)[0]
predicted_rooms_1 = model_rooms.predict(segment_1_features)[0]

profit_segment_1 = (
    predicted_dining_1 * profit_margin_dining + 
    predicted_spa_1 * profit_margin_spa + 
    predicted_rooms_1 * profit_margin_rooms
)

profit_segment_1_rounded = round(profit_segment_1)
profit_segment_1_rounded


315

In [16]:
segment_2_features = pd.DataFrame({
    'Season': [1], 
    'Brand': [1], 
    'Age': [40],
    'Income': [70000],
    'Gender': [1] 
})

segment_3_features = pd.DataFrame({
    'Season': [1], 
    'Brand': [1], 
    'Age': [50],
    'Income': [80000],
    'Gender': [1] 
})

predicted_dining_2 = model_dining.predict(segment_2_features)[0]
predicted_spa_2 = model_spa.predict(segment_2_features)[0]
predicted_rooms_2 = model_rooms.predict(segment_2_features)[0]

predicted_dining_3 = model_dining.predict(segment_3_features)[0]
predicted_spa_3 = model_spa.predict(segment_3_features)[0]
predicted_rooms_3 = model_rooms.predict(segment_3_features)[0]

profit_segment_2 = (
    predicted_dining_2 * profit_margin_dining + 
    predicted_spa_2 * profit_margin_spa + 
    predicted_rooms_2 * profit_margin_rooms
)

profit_segment_3 = (
    predicted_dining_3 * profit_margin_dining + 
    predicted_spa_3 * profit_margin_spa + 
    predicted_rooms_3 * profit_margin_rooms
)

targeting_costs = {
    "Segment 1": 200,
    "Segment 2": 300,
    "Segment 3": 500
}

net_profit_segment_1 = profit_segment_1 - targeting_costs["Segment 1"]
net_profit_segment_2 = profit_segment_2 - targeting_costs["Segment 2"]
net_profit_segment_3 = profit_segment_3 - targeting_costs["Segment 3"]

best_segment = max(
    ("Segment 1", net_profit_segment_1),
    ("Segment 2", net_profit_segment_2),
    ("Segment 3", net_profit_segment_3),
    key=lambda segment: segment[1]
)

best_segment[0], round(best_segment[1])


('Segment 2', 150)

In [17]:
train_data['Age_squared'] = train_data['Age'] ** 2
train_data['Income_squared'] = train_data['Income'] ** 2
test_data['Age_squared'] = test_data['Age'] ** 2
test_data['Income_squared'] = test_data['Income'] ** 2

updated_features = ['Season', 'Brand', 'Age', 'Income', 'Gender', 'Age_squared', 'Income_squared']

model_dining_updated = LinearRegression()
model_dining_updated.fit(train_data[updated_features], train_data[target_dining])
predictions_dining_updated = model_dining_updated.predict(test_data[updated_features])

model_spa_updated = LinearRegression()
model_spa_updated.fit(train_data[updated_features], train_data[target_spa])
predictions_spa_updated = model_spa_updated.predict(test_data[updated_features])

model_rooms_updated = LinearRegression()
model_rooms_updated.fit(train_data[updated_features], train_data[target_rooms])
predictions_rooms_updated = model_rooms_updated.predict(test_data[updated_features])

total_predicted_spending_updated = predictions_dining_updated + predictions_spa_updated + predictions_rooms_updated

rmse_total_updated = sqrt(mean_squared_error(test_data[target_total_spending], total_predicted_spending_updated))
rmse_total_updated_rounded = round(rmse_total_updated)
rmse_total_updated_rounded


211

In [18]:
segment_1_features['Age_squared'] = segment_1_features['Age'] ** 2
segment_1_features['Income_squared'] = segment_1_features['Income'] ** 2

segment_2_features['Age_squared'] = segment_2_features['Age'] ** 2
segment_2_features['Income_squared'] = segment_2_features['Income'] ** 2

segment_3_features['Age_squared'] = segment_3_features['Age'] ** 2
segment_3_features['Income_squared'] = segment_3_features['Income'] ** 2


predicted_dining_1_updated = model_dining_updated.predict(segment_1_features)[0]
predicted_spa_1_updated = model_spa_updated.predict(segment_1_features)[0]
predicted_rooms_1_updated = model_rooms_updated.predict(segment_1_features)[0]
profit_segment_1_updated = (
    predicted_dining_1_updated * profit_margin_dining + 
    predicted_spa_1_updated * profit_margin_spa + 
    predicted_rooms_1_updated * profit_margin_rooms
)
net_profit_segment_1_updated = profit_segment_1_updated - targeting_costs["Segment 1"]

predicted_dining_2_updated = model_dining_updated.predict(segment_2_features)[0]
predicted_spa_2_updated = model_spa_updated.predict(segment_2_features)[0]
predicted_rooms_2_updated = model_rooms_updated.predict(segment_2_features)[0]
profit_segment_2_updated = (
    predicted_dining_2_updated * profit_margin_dining + 
    predicted_spa_2_updated * profit_margin_spa + 
    predicted_rooms_2_updated * profit_margin_rooms
)
net_profit_segment_2_updated = profit_segment_2_updated - targeting_costs["Segment 2"]

predicted_dining_3_updated = model_dining_updated.predict(segment_3_features)[0]
predicted_spa_3_updated = model_spa_updated.predict(segment_3_features)[0]
predicted_rooms_3_updated = model_rooms_updated.predict(segment_3_features)[0]
profit_segment_3_updated = (
    predicted_dining_3_updated * profit_margin_dining + 
    predicted_spa_3_updated * profit_margin_spa + 
    predicted_rooms_3_updated * profit_margin_rooms
)
net_profit_segment_3_updated = profit_segment_3_updated - targeting_costs["Segment 3"]

optimal_segment_updated, net_profit_optimal_updated = max(
    ("Segment 1", net_profit_segment_1_updated),
    ("Segment 2", net_profit_segment_2_updated),
    ("Segment 3", net_profit_segment_3_updated),
    key=lambda segment: segment[1]
)

optimal_segment_updated, round(net_profit_optimal_updated)


('Segment 2', 182)